In [1]:
# https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import statsmodels.api as sm
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
#Authentication - without user
cid = 'e1d252163c854439adf89ee1872895ef'
secret = '196af4bb9d674ad7942fb1c2f57f55fb'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
playlist_link = 'https://open.spotify.com/playlist/3Q2GsAzMUmMcIV2TGNqleu?si=9801a625a0d64160'
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [4]:
# creating dataframe
dataframe = {'Artist' : []}
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #Name
    artist_name = track["track"]["artists"][0]["name"]
    if artist_name == 'Taylor Swift':
        dataframe['Artist'].append(1)
    else:
        dataframe['Artist'].append(0)
        
    for keys, values in sp.audio_features(track["track"]["uri"])[0].items():
        if type(values) == float or type(values) == int:
            
            if keys not in dataframe:
                dataframe[keys] = []
            dataframe[keys].append(values)


In [5]:
df = pd.DataFrame(data=dataframe)
df = df.drop(columns=['mode'])
df

,Artist,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1,0.652,0.802,7,-6.114,0.1810,0.08710,0.000006,0.1480,0.295,170.157,211933,4
1,0,0.654,0.760,0,-3.669,0.0450,0.07970,0.000000,0.2990,0.410,99.945,200787,4
2,0,0.776,0.875,4,-5.535,0.0575,0.03430,0.000000,0.3140,0.849,111.988,211786,4
3,0,0.696,0.893,9,-2.963,0.0760,0.06620,0.000000,0.4730,0.573,97.954,214013,4
4,0,0.645,0.878,5,-3.208,0.1130,0.29400,0.000000,0.0760,0.862,82.324,227027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.588,0.749,11,-4.323,0.0580,0.00370,0.000000,0.0813,0.397,130.093,174600,4
96,1,0.359,0.543,7,-7.582,0.0919,0.49200,0.000016,0.1180,0.453,68.534,221307,4
97,1,0.777,0.658,0,-5.191,0.0540,0.07670,0.000000,0.0901,0.633,110.048,190360,4
98,0,0.620,0.805,5,-5.538,0.0417,0.00564,0.000092,0.2850,0.586,122.062,204764,4


In [6]:
# create the actual model
features = df.drop(columns=['Artist'])
labels = df['Artist']

train_vectors, test_vectors, train_labels, test_labels = train_test_split(features, labels, train_size = 0.75, random_state = 314159)

In [7]:
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
result = logit_model.fit()
print(result.summary() )

         Current function value: 0.352940
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                 Artist   No. Observations:                   75
Model:                          Logit   Df Residuals:                       62
Method:                           MLE   Df Model:                           12
Date:                Wed, 20 Apr 2022   Pseudo R-squ.:                  0.2668
Time:                        10:13:37   Log-Likelihood:                -26.470
converged:                      False   LL-Null:                       -36.102
Covariance Type:            nonrobust   LLR p-value:                   0.08240
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -92.9582   5.28e+04     -0.002      0.999   -1.03e+05    1.03e+05
danceability         4.2344      4.521      0.9

/Users/mariapacifico/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [8]:
predict = result.predict(sm.add_constant(test_vectors))
final_predict = []
for i in predict:
    if i < 0.5:
        final_predict.append(0)
    else:
        final_predict.append(1)
accuracy_score(test_labels, final_predict) # accuracy score

0.84

In [13]:
def create_dataframe(playlist_link, artist):
    # create spotipy 
    cid = 'e1d252163c854439adf89ee1872895ef'
    secret = '196af4bb9d674ad7942fb1c2f57f55fb'
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
    
    # retrive data
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]
    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    
    # create the data frame
    dataframe = {'Artist' : []}
    for track in sp.playlist_tracks(playlist_URI)["items"]:
        
        # add artist
        artist_name = track["track"]["artists"][0]["name"]
        if artist_name == artist:
            dataframe['Artist'].append(1)
        else:
            dataframe['Artist'].append(0)

        # add the rest of features
        for keys, values in sp.audio_features(track["track"]["uri"])[0].items():
            if type(values) == float or type(values) == int:

                if keys not in dataframe:
                    dataframe[keys] = []
                dataframe[keys].append(values)
    
    # extract features and labels
    features = df.drop(columns=['Artist'])
    labels = df['Artist']
    return features, labels
    
def playlist_logit_model(playlist_link, artist):
    features, labels = create_dataframe(playlist_link, artist)
    
    # test and train
    train_vectors, test_vectors, train_labels, test_labels = train_test_split(features, labels, train_size = 0.75, random_state = 314159)
    
    logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
    result = logit_model.fit()
    
    return result

In [16]:
create_dataframe('https://open.spotify.com/playlist/3Q2GsAzMUmMcIV2TGNqleu?si=9801a625a0d64160', 'Tay')

(    danceability  energy  key  loudness  speechiness  acousticness  \
 0          0.652   0.802    7    -6.114       0.1810       0.08710   
 1          0.654   0.760    0    -3.669       0.0450       0.07970   
 2          0.776   0.875    4    -5.535       0.0575       0.03430   
 3          0.696   0.893    9    -2.963       0.0760       0.06620   
 4          0.645   0.878    5    -3.208       0.1130       0.29400   
 ..           ...     ...  ...       ...          ...           ...   
 95         0.588   0.749   11    -4.323       0.0580       0.00370   
 96         0.359   0.543    7    -7.582       0.0919       0.49200   
 97         0.777   0.658    0    -5.191       0.0540       0.07670   
 98         0.620   0.805    5    -5.538       0.0417       0.00564   
 99         0.292   0.574    0    -9.375       0.0401       0.12000   
 
     instrumentalness  liveness  valence    tempo  duration_ms  time_signature  
 0           0.000006    0.1480    0.295  170.157       211933   